<a href="https://colab.research.google.com/github/Gongsoo/Python3/blob/main/study/paper/papering111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd "/content/drive/MyDrive/paper"

/content/drive/MyDrive/paper


In [3]:
#!unzip -qq "/content/drive/MyDrive/paper/papering.zip"

In [4]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, SubsetRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [6]:
CFG = {
    'IMG_SIZE':256,
    'EPOCHS':5,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

In [7]:
def seed_everything(seed):
    deterministic = False
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    if deterministic :
      torch.backends.cudnn.deterministic = True
      torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

In [8]:
all_img_list = glob.glob('/content/drive/MyDrive/paper/train/*/*')

In [9]:
train = pd.DataFrame(columns=['img_path', 'label'])
train['img_path'] = all_img_list
train['label'] = train['img_path'].apply(lambda x : str(x).split('/')[-2])

In [10]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])

In [11]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [12]:
def calculate_norm(dataset,test):
    if test :
      mean_ = np.array([np.mean(x.numpy(), axis=(1, 2)) for x in tqdm(dataset)])
    else :
      mean_ = np.array([np.mean(x.numpy(), axis=(1, 2)) for x, _ in tqdm(dataset)])
    # r, g, b 채널에 대한 각각의 평균 산출
    mean_r = mean_[:, 0].mean()/255.
    mean_g = mean_[:, 1].mean()/255.
    mean_b = mean_[:, 2].mean()/255.

    if test :
      std_ = np.array([np.std(x.numpy(), axis=(1, 2)) for x in tqdm(dataset)])
    else :
      std_ = np.array([np.std(x.numpy(), axis=(1, 2)) for x, _ in tqdm(dataset)])
    # r, g, b 채널에 대한 각각의 표준편차 산출
    std_r = std_[:, 0].mean()/255.
    std_g = std_[:, 1].mean()/255.
    std_b = std_[:, 2].mean()/255.
    
    return (mean_r, mean_g, mean_b), (std_r, std_g, std_b)
     

In [13]:
train_resize = A.Compose([
                            ToTensorV2()
                            ])

test_resize = A.Compose([
                            ToTensorV2()
                            ])

In [14]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_resize)

In [15]:
mean_train, std_train = calculate_norm(train_dataset,False)

  0%|          | 0/3457 [00:00<?, ?it/s]

  0%|          | 0/3457 [00:00<?, ?it/s]

In [16]:
train_transform = A.Compose([
                            A.Normalize(max_pixel_value=255.0, always_apply=False, p=1.0),
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.OneOf([
                                A.HorizontalFlip(p=1),
                                A.ShiftScaleRotate(shift_limit = 0.07, scale_limit=0.07, rotate_limit=0,p=1)
                            ], p=1),
                            A.OneOf([
                                A.CoarseDropout(p=1, max_holes=10, max_height=10, max_width=10, min_holes=1, min_height=5, min_width=5),
                                A.GaussNoise(var_limit=(0.25),p=1)
                            ],p=1),
                            ToTensorV2()
                            ])

val_transform = A.Compose([
                            A.Normalize(max_pixel_value=255.0, always_apply=False, p=1.0),
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.OneOf([
                                A.HorizontalFlip(p=1),
                                A.ShiftScaleRotate(shift_limit = 0.07, scale_limit=0.07, rotate_limit=0,p=1)
                            ], p=1),
                            A.OneOf([
                                A.CoarseDropout(p=1, max_holes=10, max_height=10, max_width=10, min_holes=1, min_height=5, min_width=5),
                                A.GaussNoise(var_limit=(0.25),p=1)
                            ],p=1),
                            ToTensorV2()
                            ])

In [17]:
train_dataset = CustomDataset(train['img_path'].values,train['label'].values, train_transform)

In [18]:
class Network(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(Network, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        x = F.log_softmax(x, dim=1)
        return x

In [19]:
def train_model(model, optimizer, scheduler, device):

    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    fold_train_loss = 0
    fold_val_loss = 0
    fold_val_score = 0
    for epoch in range(1,CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
        fold_train_loss+=_train_loss
        fold_val_loss+=_val_loss
        fold_val_score+=_val_score
    if scheduler is not None:
        scheduler.step(_val_score)
        
        
    
    return fold_train_loss/CFG["EPOCHS"],fold_val_loss/CFG["EPOCHS"],fold_val_score/CFG["EPOCHS"]

In [20]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='macro')
    
    return _val_loss, _val_score

In [21]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = CFG['SEED'])
best_model = None
best_score = 0
for fold, (train_idx, val_idx) in enumerate(skf.split(train['img_path'],train['label'])) :
    

    
    train_subsampler = SubsetRandomSampler(train_idx)
    val_subsampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], sampler=train_subsampler,num_workers=0)
    val_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], sampler=val_subsampler, num_workers=0)

    model = Network().to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.33, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)
    
    fold_train_loss, fold_val_loss, fold_val_score = train_model(model,optimizer,scheduler,device)
    print(f'kfold [{fold+1}], Fold Train Loss : [{fold_train_loss:.5f}] Fold Val Loss : [{fold_val_loss:.5f}] Fold Val macro F1 Score : [{fold_val_score:.5f}]')

    if best_score < fold_val_score :
      best_model = model
      best_score = fold_val_score

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 140MB/s] 


  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

Epoch [1], Train Loss : [1.60271] Val Loss : [1.21321] Val Weighted F1 Score : [0.34296]


  0%|          | 0/87 [00:00<?, ?it/s]

...

  0%|          | 0/22 [00:00<?, ?it/s]

Epoch [4], Train Loss : [0.74473] Val Loss : [0.95145] Val Weighted F1 Score : [0.43613]


  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

Epoch [5], Train Loss : [0.65622] Val Loss : [0.86867] Val Weighted F1 Score : [0.48147]
kfold [5], Fold Train Loss : [0.98588] Fold Val Loss : [1.01894] Fold Val macro F1 Score : [0.43154]


In [22]:
torch.save(best_model, f'/content/drive/MyDrive/paper/crossval_epoch{CFG["EPOCHS"]}_batch{CFG["BATCH_SIZE"]}_size{CFG["IMG_SIZE"]}_aug_model.pt')

In [36]:
torch.save(model, f'/content/drive/MyDrive/paper/crossval_epoch{CFG["EPOCHS"]}_batch{CFG["BATCH_SIZE"]}_size{CFG["IMG_SIZE"]}_aug_modelv2.pt')

In [23]:
test = pd.read_csv('./test.csv')

In [24]:
test_dataset = CustomDataset(test['img_path'].values, None, test_resize)

In [25]:
mean_test, std_test = calculate_norm(test_dataset,True)

  0%|          | 0/792 [00:00<?, ?it/s]

  0%|          | 0/792 [00:00<?, ?it/s]

In [26]:
test_transform = A.Compose([
                            A.Normalize(mean_test, std_test, max_pixel_value=255.0, always_apply=False, p=1.0),
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            ToTensorV2()
                            ])

In [27]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)

In [28]:
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [29]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    return preds

In [37]:
preds = inference(model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

In [38]:
submit = pd.read_csv('./sample_submission.csv')

In [39]:
submit['label'] = preds

In [40]:
submit.loc[submit['label'] == 0, 'label'] = '가구수정'
submit.loc[submit['label'] == 1, 'label'] = '걸레받이수정'
submit.loc[submit['label'] == 2, 'label'] = '곰팡이'
submit.loc[submit['label'] == 3, 'label'] = '꼬임'
submit.loc[submit['label'] == 4, 'label'] = '녹오염'
submit.loc[submit['label'] == 5, 'label'] = '들뜸'
submit.loc[submit['label'] == 6, 'label'] = '면불량'
submit.loc[submit['label'] == 7, 'label'] = '몰딩수정'
submit.loc[submit['label'] == 8, 'label'] = '반점'
submit.loc[submit['label'] == 9, 'label'] = '석고수정'
submit.loc[submit['label'] == 10, 'label'] = '오염'
submit.loc[submit['label'] == 11, 'label'] = '오타공'
submit.loc[submit['label'] == 12, 'label'] = '울음'
submit.loc[submit['label'] == 13, 'label'] = '이음부불량'
submit.loc[submit['label'] == 14, 'label'] = '창틀,문틀수정'
submit.loc[submit['label'] == 15, 'label'] = '터짐'
submit.loc[submit['label'] == 16, 'label'] = '틈새과다'
submit.loc[submit['label'] == 17, 'label'] = '피스'
submit.loc[submit['label'] == 18, 'label'] = '훼손'

In [41]:
submit.to_csv(f'./crossval_epoch{CFG["EPOCHS"]}_batch{CFG["BATCH_SIZE"]}_size{CFG["IMG_SIZE"]}_aug_model_submitv2.csv', index=False)